In [ ]:
from pathlib import Path
import csv
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [ ]:
fmc = Path("./FakeMusicCaps/")

In [ ]:
data_sets = [
    "MusicCaps",
    "MusicGen_medium",
    "SunoCaps",
    "audioldm2",
    "musicldm",
    "mustango",
    "stable_audio_open",
]

In [ ]:
import os

for ds in data_sets:
    print(ds)
    print(len(os.listdir(f'./FakeMusicCaps/{ds}')))
    print()

In [ ]:

for folder in data_sets:
    folder_path = fmc / folder
    for music in folder_path.glob('*.wav'):
        y, sr = librosa.load(music)
        plt.figure(figsize=(14, 5))
        librosa.display.waveshow(y, sr=sr)
        plt.show()
        break

In [ ]:
for folder in data_sets:
    folder_path = fmc / folder
    for music in folder_path.glob('*.wav'):
        y, sr = librosa.load(music)
        mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
        spectrogram = np.abs(mel_signal)
        power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
        plt.figure(figsize=(8, 7))
        librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma')
        plt.colorbar(label='dB')
        plt.title('Mel-Spectrogram (dB)', fontdict=dict(size=18))
        plt.xlabel('Time', fontdict=dict(size=15))
        plt.ylabel('Frequency', fontdict=dict(size=15))
        plt.show()
        break

### I need to split the data into test/train

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use('agg')

spectrogram_folder = Path('./Spectrograms/')
spectrogram_folder.mkdir(exist_ok=True)

for folder in data_sets:
    count = 0
    folder_path = fmc / folder
    songs = list(folder_path.glob('*.wav'))
    for music in folder_path.glob('*.wav'):
        count += 1
        # print(count, song)
        destination_folder = Path(spectrogram_folder / folder)
        destination_folder.mkdir(exist_ok=True)
        y, sr = librosa.load(music)
        mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
        spectrogram = np.abs(mel_signal)
        power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
        fig = plt.figure(figsize=(8, 7))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma')
        fig.savefig(destination_folder / f"{music.stem}.png")
        plt.close()

In [ ]:
import gc
gc.collect()

In [ ]:
import os
import shutil

spectrogram_folder = Path('./Spectrograms/Spectrograms/')

binary_spectrogram_folder = Path('./BinaryClassifiedSpectrograms/')
binary_spectrogram_folder.mkdir(exist_ok=True)

destination_folder = Path(binary_spectrogram_folder / "deepfake")
destination_folder.mkdir(exist_ok=True)

for folder in data_sets:
    if folder != 'SunoCaps' and folder != 'MusicCaps':
        folder_path = spectrogram_folder / folder
        max_samples = len(os.listdir(spectrogram_folder / "MusicCaps")) * 0.2
        count = 0
        for spectrogram in folder_path.glob('*.png'):
            if count > max_samples:
                break
            count += 1
            shutil.copyfile(spectrogram, destination_folder / f"{spectrogram.stem}_{folder}.png")

human_destination = Path(binary_spectrogram_folder / "human")
human_destination.mkdir(exist_ok=True)

folder_path = spectrogram_folder / "MusicCaps"
for spectrogram in folder_path.glob('*.png'):
    shutil.copyfile(spectrogram, human_destination / spectrogram.name)

In [ ]:
for folder in ["human", "deepfake"]:
    print(len(os.listdir(f'./BinaryClassifiedSpectrograms/{folder}')))